In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_horizontal_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,65318.38,-0.10,-0.07,-0.12,-0.05,-0.00,-0.01,0.00
3,-0.10,11384.46,-0.02,0.04,0.01,0.01,0.00,-0.00
4,-0.07,-0.02,1873.94,-0.03,-0.00,0.00,-0.00,0.00
5,-0.12,0.04,-0.03,299.78,0.00,-0.00,0.00,-0.00
6,-0.05,0.01,-0.00,0.00,46.74,-0.00,0.00,0.00
7,-0.00,0.01,0.00,-0.00,-0.00,6.96,-0.00,-0.00
8,-0.01,0.00,-0.00,0.00,0.00,-0.00,1.04,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.14


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00000,-0.00001,-0.00003,-0.00003,-0.00001,-0.00002,0.00001
3,-0.00000,1.00000,-0.00000,0.00002,0.00001,0.00002,0.00000,-0.00001
4,-0.00001,-0.00000,1.00000,-0.00004,-0.00001,0.00004,-0.00002,0.00001
5,-0.00003,0.00002,-0.00004,1.00000,0.00002,-0.00000,0.00001,-0.00002
6,-0.00003,0.00001,-0.00001,0.00002,1.00000,-0.00001,0.00002,0.00003
7,-0.00001,0.00002,0.00004,-0.00000,-0.00001,1.00000,-0.00001,-0.00001
8,-0.00002,0.00000,-0.00002,0.00001,0.00002,-0.00001,1.00000,0.00001
9,0.00001,-0.00001,0.00001,-0.00002,0.00003,-0.00001,0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.26399509426894546

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.33662095e+08 1.58726476e+07 4.18176501e+05 1.02761247e+04
 2.34253880e+02 4.62448454e+00 7.79081646e-02 1.63532354e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999662,-0.025652,-0.004080,-0.000654,-0.000102,-0.000015,-0.000002,-2.970133e-07
1,0.025523,0.999264,-0.028279,-0.004380,-0.000682,-0.000101,-0.000015,-2.095210e-06
2,0.004774,0.028003,0.999041,-0.032938,-0.004980,-0.000738,-0.000111,-1.484557e-05
3,0.000917,0.005248,0.032570,0.998636,-0.040027,-0.005783,-0.000862,-1.170623e-04
4,0.000179,0.001020,0.006209,0.039488,0.997879,-0.050819,-0.007358,-9.934417e-04
5,0.000035,0.000201,0.001222,0.007655,0.049830,0.995775,-0.076086,-9.957415e-03
6,0.000008,0.000044,0.000270,0.001688,0.010830,0.073587,0.988025,-1.351735e-01
7,0.000002,0.000012,0.000074,0.000464,0.002974,0.019996,0.134027,9.907714e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00033760145306804823,
 0.0007356619787660668,
 0.0009590784638388028,
 0.0013636753007070235,
 0.0021205517560179965,
 0.0042246098494865825,
 0.0119752345695614,
 0.009228594136348689]